# Spectrogram Generator

This part of the project use Deep Learning to automatize feature extraction. For that propourse we convert waveforms to spectrograms images. This notebook can be avoid beacuse it data is provided in the file "spectrograms.zip"

<a href="https://colab.research.google.com/drive/1hP9gDx5AkAymNtzz9ZdeJgjlLtmZ0C-p#scrollTo=3-ruJnWwzFxn">Colab</a>



In [1]:
from scipy import signal
from scipy.fft import fftshift
import matplotlib.pyplot as plt
import pandas as pd
from Scripts.datosadash import datosADASH
import re
import os
import pandas as pd
import numpy as np
import pywt

ModuleNotFoundError: No module named 'pywt'

In [15]:
# Now we want to get a dataframe with the relation between every spectrogram
# and the label. This label is the diagnostic made be the analyst. We import
# the csv "diagnosticos_total.csv" where this information is saved. Then we create
# an empty dataframe to write the information of the spectrogram. These spectrogram
# will be merged with the diagnostics csv.

diag = pd.read_csv('Data/diagnosis.csv')
spectro_diag = pd.DataFrame(columns=['file','time'])

diag.head()

,Unnamed: 0,Nombre,diagnostico,Id_Estado_Activo_fixed,time,max_mms,machine
0,0,S3-Ventilador M38,Motor: Activo en buen estado.\n Ventilador: S...,2.0,2019-06-18 12:38:26.205,9.2617,S3-M38
1,1,S3-Ventilador M38,Motor: Activo en buen estado.\n Ventilador: S...,2.0,2019-06-18 12:38:41.249,6.4828,S3-M38
2,2,S3-Ventilador M38,Motor: Activo en buen estado.\n Ventilador: S...,2.0,2019-06-18 12:38:57.866,7.4407,S3-M38
3,3,S3-Ventilador M38,Motor: Activo en buen estado.\n Ventilador: S...,2.0,2019-06-18 12:39:08.660,6.9886,S3-M38
4,4,S3-Ventilador M38,Motor: Activo en buen estado.\n Ventilador: S...,2.0,2019-06-18 12:39:20.397,7.3701,S3-M38


In [ ]:
%%time
# We select the folder where waveforms are and get machine, points and files
folder = datosADASH('Data/Export')
machines = folder.getmachines()
points = folder.getpoints()
files = folder.getfiles()

# Spectrogrograms will be created with Scypy.
for machine, points_iter in points.items():
    filteredmachines = list(filter(lambda x: machine in x, files))
    for iteration, point in enumerate(points_iter):
        acc_spec_text = "Aceleracion - Forma"
        filteredpoints = list(filter(lambda x: point in x, filteredmachines))
        filteredcsv = list(filter(lambda x:".csv" in x, filteredpoints))
                        
        # Get files by type of measurement
        accspec = list(filter(lambda x:acc_spec_text in x, filteredcsv))
                
                
        for filetime in accspec:
            forma = pd.read_csv(filetime, skiprows=1,
                                        delimiter=";", encoding = "ISO-8859-1")
            forma = forma.rename(columns={'time[ms]':'time',
                                                  ' amplitude[g]':'amplitude'})
            date = pd.read_csv(filetime, nrows=0,
                                        delimiter=";", encoding = "ISO-8859-1").columns[1][6:]
            fs = 12000
            fig,ax = plt.subplots(1)
            fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
            ax.axis('off')
            # Create the spetrogrmas
            f, t, Sxx = signal.spectrogram(forma['amplitude'], fs)
            plt.pcolormesh(t, f, Sxx)
            ax.axis('tight')
            ax.axis('off')
            date_sub = re.sub('[^A-Za-z0-9]+','', date)
                    
            filename = str(date_sub) + "_" + str(machine) + "_" + str(point)
            folder = "Data/Spectrograms"
            filename = os.path.join(folder, filename)
            fig.savefig(filename, transparent=True, pad_inches=0.0,
                                figsize=(96/96, 96/96), dpi=96)
            plt.close()
            
            spectro_diag = spectro_diag.append({"file":filename,
                                                "time":date,
                                                "machine":machine
                                                }, ignore_index=True)

In [ ]:
# Now we need to create a dataframe where images and status 
# are correlated.

def sorted_index_by_time(df):
    """Function for sorting values
    by time and set it to index"""
    df = df.sort_values(['time'])
    df['time'] = pd.to_datetime(df['time'])
    return df

diag = sorted_index_by_time(diag)
spectro_diag = sorted_index_by_time(spectro_diag)

merged_diagnostic = pd.merge_asof(spectro_diag, diag, on="time", by=['machine'],
                  tolerance=pd.Timedelta('10 days'), direction='nearest')

merged_diagnostic[['file', 'time', 'machine','Nombre', 'diagnostico',
       'Id_Estado_Activo_fixed']]

In [15]:
# For deep learning will need to move the images to
# one folder for the two labels. 


merged_diagnostic.file = merged_diagnostic.file + '.png'
merged_diagnostic['filecut'] = merged_diagnostic['file'].str[18:]


for index, row in merged_diagnostic.iterrows():
    filecut = row.filecut
    file = row.file
    if row.Id_Estado_Activo_fixed == 1.0:
        filename = "Data/Spectrograms/class_a/"
        final_filename = filename + filecut
        os.rename(file, final_filename)
    if row.Id_Estado_Activo_fixed > 1:
        filename = "Data/Spectrograms/class_b/"
        final_filename = filename + filecut
        os.rename(file, final_filename)

## Wavelets

We also are going to use wavelets and power spectrums. AMPLIAR BIBLIOGRAFIA

In [9]:
forma = pd.read_csv('Data/formaonda.csv', skiprows=1,
                                        delimiter=";", encoding = "ISO-8859-1")
forma = forma.rename(columns={'time[ms]':'time',' amplitude[g]':'amplitude'})

,time,amplitude,
0,0.00000,-0.01333,
1,0.06104,0.02847,
2,0.12207,0.05585,
3,0.18311,0.07309,
4,0.24414,0.07005,
...,...,...,...
8187,499.69000,-0.26479,
8188,499.76000,-0.31094,
8189,499.82000,-0.38142,
8190,499.88000,-0.38712,


In [ ]:
%%time
# We select the folder where waveforms are and get machine, points and files
folder = datosADASH('Data/Export')
machines = folder.getmachines()
points = folder.getpoints()
files = folder.getfiles()

# Wavelets and power spectrums are generated with pywt
for machine, points_iter in points.items():
    filteredmachines = list(filter(lambda x: machine in x, files))
    for iteration, point in enumerate(points_iter):
        acc_spec_text = "Aceleracion - Forma"
        filteredpoints = list(filter(lambda x: point in x, filteredmachines))
        filteredcsv = list(filter(lambda x:".csv" in x, filteredpoints))
                        
        # Get files by type of measurement
        accspec = list(filter(lambda x:acc_spec_text in x, filteredcsv))
                
                
        for filetime in accspec:
            forma = pd.read_csv(filetime, skiprows=1,
                                        delimiter=";", encoding = "ISO-8859-1")
            forma = forma.rename(columns={'time[ms]':'time',
                                                  ' amplitude[g]':'amplitude'})
            date = pd.read_csv(filetime, nrows=0,
                                        delimiter=";", encoding = "ISO-8859-1").columns[1][6:]
            
            
            time = forma['time']
            sst = forma['amplitude']
            dt = time[1] - time[0]

            wavelet = 'cmor1.5-1.0'
            scales = np.arange(1, 128)

            [cfs, frequencies] = pywt.cwt(sst, scales, wavelet, dt)
            power = (abs(cfs)) ** 2

            period = 1. / frequencies
            levels = [0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8]
            dpi = 96


            f, ax = plt.subplots(1, figsize=(400/dpi, 400/dpi))
            f.subplots_adjust(left=0,right=1,bottom=0,top=1)
            ax.contourf(time, np.log2(period), np.log2(power), np.log2(levels),
                        extend='both')
            ax.invert_yaxis()
            ax.axis('off')
            
            
            date_sub = re.sub('[^A-Za-z0-9]+','', date)    
             
            filename = str(date_sub) + "_" + str(machine) + "_" + str(point)
            
            folder = "Data/Wavelets"
            filename = os.path.join(folder, filename)
            
            
            f.savefig(filename, transparent=True, pad_inches=0.0,
                                figsize=(96/96, 96/96), dpi=96)
            
            plt.close()
            
            
            spectro_diag = spectro_diag.append({"file":filename,
                                                "time":date,
                                                "machine":machine
                                                }, ignore_index=True)

In [ ]:
# Now we need to create a dataframe where images and status 
# are correlated.

def sorted_index_by_time(df):
    """Function for sorting values
    by time and set it to index"""
    df = df.sort_values(['time'])
    df['time'] = pd.to_datetime(df['time'])
    return df

diag = sorted_index_by_time(diag)
spectro_diag = sorted_index_by_time(spectro_diag)

merged_diagnostic = pd.merge_asof(spectro_diag, diag, on="time", by=['machine'],
                  tolerance=pd.Timedelta('10 days'), direction='nearest')

merged_diagnostic[['file', 'time', 'machine','Nombre', 'diagnostico',
       'Id_Estado_Activo_fixed']]

In [ ]:
# Move files to separate labeled folders


merged_diagnostic.file = merged_diagnostic.file + '.png'
merged_diagnostic['filecut'] = merged_diagnostic['file'].str[18:]


for index, row in merged_diagnostic.iterrows():
    filecut = row.filecut
    file = row.file
    if row.Id_Estado_Activo_fixed == 1.0:
        filename = "Data/Spectrograms/Wavelets/class_a/"
        final_filename = filename + filecut
        os.rename(file, final_filename)
    if row.Id_Estado_Activo_fixed > 1:
        filename = "Data/Spectrograms/Wavelets/class_b/"
        final_filename = filename + filecut
        os.rename(file, final_filename)